In [114]:
import pydot
import matplotlib.pyplot as plt
import time
import math
import random
from sklearn.manifold import MDS
import numpy as np
import plotly.graph_objs as go
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap


In [115]:
def get_data_step6(dot_file_path): #nodes,edges,node_list,edge_list,weights
    '''Returns the nodes, edges (both in pydot format), a list with all the nodes and another with all the edges, and also a dic with (edge):weigth
    '''
     #= 'Datasets/LesMiserables.dot'
    with open(dot_file_path, 'r') as dot_file:
        dot_data = dot_file.read()
    graph = pydot.graph_from_dot_data(dot_data)

    nodes = graph[0].get_nodes()
    edges = graph[0].get_edges()
    
    node_list = [node.get_name() for node in nodes]
    edge_list = [(edge.get_source(), edge.get_destination()) for edge in edges]
    weights = {}
    if edges[0].get_attributes()!={}:
        name= str(list(edges[0].get_attributes().keys())[0])
        weights = {(edge.get_source(), edge.get_destination()):int(edge.get_attributes()[name]) for edge in edges}

    return nodes,edges,node_list,edge_list,weights

In [107]:
#nodes,edges,node_list,edge_list,weights = get_data_step6('Datasets/LesMiserables.dot')

#print("nodes", nodes)
#print("edges", nodes)
#print("node_list", node_list)
#print("edges_list", edge_list)
#print("weights", weights)

# Floyd Warshall

In [116]:
def floyd_warshall(node_list, edge_list, weights):
    num_nodes = len(node_list)
    
    distances = np.full((num_nodes, num_nodes), np.inf)
    np.fill_diagonal(distances, 0)

    for (u, v), weight in weights.items():
        i = node_list.index(u)
        j = node_list.index(v)
        distances[i, j] = weight
        distances[j, i] = weight  # If the graph is undirected

    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                distances[i, j] = min(distances[i, j], distances[i, k] + distances[k, j])

    return distances

distances = floyd_warshall(node_list, edge_list, weights)
#print(type(distances))

# T-SNE

In [117]:
def plot_tsne(distances, node_list, edge_list, perplexity=20, learning_rate=100, n_iter=1000, interactive=False):
    tsne = TSNE(n_components=3, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter, init='random', metric='precomputed')
    embeddings = tsne.fit_transform(distances)
    #KMeans clustering
    kmeans = KMeans(n_clusters=8)  # You can adjust the number of clusters
    labels = kmeans.fit_predict(embeddings)

    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=labels, cmap='viridis', s=50)
    #plt.scatter(embeddings[:, 0], embeddings[:, 1], cmap='viridis', s=50)

    for edge in edge_list:
        u, v = edge
        u_index = node_list.index(u)
        v_index = node_list.index(v)
        plt.plot([embeddings[u_index, 0], embeddings[v_index, 0]], 
                    [embeddings[u_index, 1], embeddings[v_index, 1]], 
                    color='gray', alpha=0.2)

    for i, node_number in enumerate(node_list):
        plt.text(embeddings[i, 0], embeddings[i, 1], node_number, color='black')

    plt.title('t-SNE Visualization of Node Similarities')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.show()

    if interactive:
        trace = go.Scatter3d(
            x=embeddings[:, 0],
            y=embeddings[:, 1],
            z=embeddings[:, 2],  
            mode='markers',
            text=node_list,
            hoverinfo='text',
            marker=dict(
                size=5,
                color=labels,  
                #color=similarities.max() - similarities,
                colorscale='Viridis',
                opacity=0.8
            )
        )

        edge_traces = []
        for edge in edge_list:
            u, v = edge
            u_index = node_list.index(u)
            v_index = node_list.index(v)
            edge_trace = go.Scatter3d(
                x=[embeddings[u_index, 0], embeddings[v_index, 0]],
                y=[embeddings[u_index, 1], embeddings[v_index, 1]],
                z=[embeddings[u_index, 2], embeddings[v_index, 2]], 
                mode='lines',
                line=dict(color='gray', width=1),
                hoverinfo='none'
            )
            edge_traces.append(edge_trace)
        
        dropdown_buttons = [
            {
                'label': f'Cluster {cluster}',
                'method': 'update',
                'args': [
                    {'selectedpoints': [i for i, label in enumerate(labels) if label == cluster]},
                    {'marker.color': ['rgba(255,0,0,1)' if lab == cluster else 'rgba(0,0,0,1)' for lab in labels]}
                ]
            }
            for cluster in np.unique(labels)
        ]

        updatemenu = [
            {
                'buttons': dropdown_buttons,
                'direction': 'down',
                'showactive': True,
            }
        ]

        layout = go.Layout(
            scene=dict(
                xaxis=dict(title='t-SNE Dimension 1'),
                yaxis=dict(title='t-SNE Dimension 2'),
                zaxis=dict(title='t-SNE Dimension 3')
            ),
            margin=dict(l=0, r=0, b=0, t=0),
            updatemenus=updatemenu
        )

        fig = go.Figure(data=[trace] + edge_traces, layout=layout)
        fig.show()

#nodes,edges,node_list,edge_list,weights = get_data_step6('Datasets/LesMiserables.dot')
#distances = floyd_warshall(node_list, edge_list, weights)
#plot_tsne(distances, node_list, edge_list, interactive=True)

# MDS


In [118]:
def plot_mds(distances, node_list, edge_list, n_components=2, interactive=False):  
    mds = MDS(n_components=n_components,max_iter=750)
    embeddings = mds.fit_transform(distances)

    kmeans = KMeans(n_clusters=8)
    labels = kmeans.fit_predict(embeddings)

    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=labels, cmap='viridis', s=50)


    for edge in edge_list:
        u, v = edge
        u_index = node_list.index(u)
        v_index = node_list.index(v)
        plt.plot([embeddings[u_index, 0], embeddings[v_index, 0]], 
                    [embeddings[u_index, 1], embeddings[v_index, 1]], 
                    color='gray', alpha=0.5)

    for i, node_number in enumerate(node_list):
        plt.text(embeddings[i, 0], embeddings[i, 1], node_number, color='black')

    plt.title('MDS Visualization of Node Similarities')
    plt.xlabel('MDS Dimension 1')
    plt.ylabel('MDS Dimension 2')
    #plt.grid(True)
    plt.show()

    if interactive:
        trace = go.Scatter3d(
            x=embeddings[:, 0],
            y=embeddings[:, 1],
            z=embeddings[:, 2],
            mode='markers',
            text=node_list,
            hoverinfo='text',
            marker=dict(
                size=5,
                color=labels,
                colorscale='Viridis',
                opacity=0.8
            )
        )
        
        edge_traces = []
        for edge in edge_list:
            u, v = edge
            u_index = node_list.index(u)
            v_index = node_list.index(v)
            edge_trace = go.Scatter3d(
                x=[embeddings[u_index, 0], embeddings[v_index, 0]],
                y=[embeddings[u_index, 1], embeddings[v_index, 1]],
                z=[embeddings[u_index, 2], embeddings[v_index, 2]],
                mode='lines',
                line=dict(color='gray', width=1),
                hoverinfo='none'
            )
            edge_traces.append(edge_trace)
        
        dropdown_buttons = [
            {
                'label': node,
                'method': 'update',
                'args': [
                    {'selectedpoints': [node_list.index(node)]},
                    {'marker.color': ['rgba(255,0,0,1)' if i == node_list.index(node) else 'rgba(0,0,0,1)' for i in range(len(node_list))]}
                ]
            }
            for node in node_list
        ]

        updatemenu = [
            {
                'buttons': dropdown_buttons,
                'direction': 'down',
                'showactive': True,
            }
        ]

        layout = go.Layout(
            scene=dict(
                xaxis=dict(title='MDS Dimension 1'),
                yaxis=dict(title='MDS Dimension 2'),
                zaxis=dict(title='MDS Dimension 3')
            ),
            margin=dict(l=0, r=0, b=0, t=0),
            updatemenus=updatemenu
        )

        fig = go.Figure(data=[trace] + edge_traces, layout=layout)
        fig.show()

#nodes, edges, node_list, edge_list, weights = get_data_step6('Datasets/LesMiserables.dot')
#distances = floyd_warshall(node_list, edge_list, weights)
#plot_mds(distances, node_list, edge_list, n_components=2, interactive=False)

# ISOMAP

In [109]:
def plot_isomap(similarities, node_list, edge_list, n_components=2):  
    isomap = Isomap(n_components=n_components, max_iter=1000)
    embeddings = isomap.fit_transform(1 - similarities)

    label_map = {label: i for i, label in enumerate(np.unique(node_list))}

    plt.figure(figsize=(10, 10))
    plt.scatter(embeddings[:, 0], embeddings[:, 1])

    for edge in edge_list:
        u, v = edge
        u_index = node_list.index(u)
        v_index = node_list.index(v)
        plt.plot([embeddings[u_index, 0], embeddings[v_index, 0]], 
                 [embeddings[u_index, 1], embeddings[v_index, 1]], 
                 color='gray', alpha=0.5)

    for i, node_number in enumerate(node_list):
        plt.annotate(node_number, (embeddings[i, 0], embeddings[i, 1]), textcoords="offset points", xytext=(0,10), ha='center')

    plt.title('Isomap Visualization of Node Similarities')
    plt.xlabel('Isomap Dimension 1')
    plt.ylabel('Isomap Dimension 2')
    plt.grid(True)
    plt.show()

#plot_isomap(similarity_matrix, node_list, edge_list, n_components=2)

# ////////////////////////////// TRASH //////////////////////////////////////////

In [119]:
def compute_similarity_matrix(node_list, edge_list, weights):
    edge_weights = {(u, v): weight for (u, v), weight in weights.items()}

    num_nodes = len(node_list)
    distances = np.zeros((num_nodes, num_nodes))
    for i, source in enumerate(node_list):
        for j, target in enumerate(node_list):
            if i != j:
                if (source, target) in edge_weights:
                    distances[i, j] = edge_weights[(source, target)]
                elif (target, source) in edge_weights:
                    distances[i, j] = edge_weights[(target, source)]
                else:
                    distances[i, j] = np.inf

    similarities = np.zeros((num_nodes, num_nodes))
    for u, v in edge_list:
        i = node_list.index(u)
        j = node_list.index(v)
        similarities[i, j] = 1 / (1 + distances[i, j])
        similarities[j, i] = similarities[i, j]  # Similarity matrix is symmetric

    np.fill_diagonal(similarities, 1) #diagonal 1

    return similarities

similarity_matrix = compute_similarity_matrix(node_list, edge_list, weights)

# print("Similarity matrix:")
# print("\t" + "\t".join(node_list))
# for i in range(len(node_list)):
#     print(node_list[i], "\t", end="")
#     for j in range(len(node_list)):
#         print("{:.2f}".format(similarity_matrix[i, j]), "\t", end="")
#     print()

## Jazz similarity matrix

In [126]:
def compute_similarity_matrix_jazz(node_list, edge_list):
    num_nodes = len(node_list)
    
    distances = np.full((num_nodes, num_nodes), np.inf)
    
    for u, v in edge_list:
        i = node_list.index(u)
        j = node_list.index(v)
        distances[i, j] = 1
        distances[j, i] = 1  
    
    similarities = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                similarities[i, j] = 1 / (1 + distances[i, j])
    
    np.fill_diagonal(similarities, 1)
    
    return similarities

# similarity_matrix = compute_similarity_matrix(node_list, edge_list)

# print("Similarity matrix:")
# print("\t" + "\t".join(node_list))
# for i in range(len(node_list)):
#     print(node_list[i], "\t", end="")
#     for j in range(len(node_list)):
#         print("{:.2f}".format(similarity_matrix[i, j]), "\t", end="")
#     print()

## League Similarity matrix

In [122]:
def compute_similarity_matrix(node_list, edge_list, weights):
    edge_weights = {(u, v): weight for (u, v), weight in weights.items()}

    num_nodes = len(node_list)
    distances = np.zeros((num_nodes, num_nodes))
    for i, source in enumerate(node_list):
        for j, target in enumerate(node_list):
            if i != j:
                if (source, target) in edge_weights:
                    distances[i, j] = edge_weights[(source, target)]
                elif (target, source) in edge_weights:
                    distances[i, j] = -edge_weights[(target, source)]  # Taking negative to convert goal difference to distance
                else:
                    distances[i, j] = np.inf

    similarities = np.zeros((num_nodes, num_nodes))
    for u, v in edge_list:
        i = node_list.index(u)
        j = node_list.index(v)
        similarities[i, j] = 1 / (1 + abs(distances[i, j]))  # Taking absolute value of distance
        similarities[j, i] = similarities[i, j]  # Similarity matrix is symmetric

    np.fill_diagonal(similarities, 1)  # Diagonal set to 1

    return similarities

#similarity_matrix = compute_similarity_matrix(node_list, edge_list, weights)

# print("Similarity matrix:")
# print("\t" + "\t".join(node_list))
# for i in range(len(node_list)):
#     print(node_list[i], "\t", end="")
#     for j in range(len(node_list)):
#         print("{:.2f}".format(similarity_matrix[i, j]), "\t", end="")
#     print()

# t-SNE (base)

In [125]:
def plot_tsne(similarities, node_list, edge_list, perplexity=20, learning_rate=300, n_iter=500):
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter)
    embeddings = tsne.fit_transform(1 - similarities)

#plot_tsne(similarity_matrix, node_list, edge_list)